<a href="https://colab.research.google.com/github/FirsMak/Spiking-Neural-Network-With-Synaptic-Plasticity/blob/main/spiking_neural_network_with_synaptic_plasticity_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Synaptic Plasticity Model

---



## torch == 1.8.0, torchtext == 0.9.0

In [ ]:
#!pip install torch==1.8.0 torchtext==0.9.0

In [ ]:
!pip install snntorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 50.8 MB/s eta 0:00:00


In [ ]:
!pip install -q youtokentome
!pip install -q datasets
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 14.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2

In [ ]:
!pip install exrex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
import torchtext
import exrex
#from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint
from torch.utils.tensorboard import SummaryWriter

import random
import math
import time
import numpy as np
#from nltk.tokenize import WordPunctTokenizer

import matplotlib
matplotlib.rcParams.update({'figure.figsize': (16, 12), 'font.size': 14})
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline
from IPython.display import clear_output

#import datasets
#import gensim.downloader as api
#from tqdm.notebook import tqdm, trange
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import snntorch

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## MY_MODEL

In [ ]:
class MyConfig():
  def __init__(self, device, is_bias, dropout, hid_dim, pf_dim, synapse_static_dim, synapse_dynamic_dim, neuron_static_dim, neuron_dynamic_dim):
    self.device = device
    self.is_bias = is_bias

    self.hid_dim = hid_dim
    self.pf_dim = pf_dim

    self.synapse_static_dim = synapse_static_dim
    self.synapse_dynamic_dim = synapse_dynamic_dim

    self.neuron_static_dim = neuron_static_dim
    self.neuron_dynamic_dim = neuron_dynamic_dim

    self.dropout = config.dropout

In [ ]:
class MyModel(nn.Module):
    def __init__(self,
                 decoder,
                 config):
        super().__init__()


        self.decoder = decoder
        self.device = config.device
        self.output_dim = config.output_dim
        self.dropout = config.dropout
        self.n_layer = config.n_layer

        self.embeddings = nn.Embedding(self.output_dim, self.hid_dim) # create embeddings for intent
        self.logits = nn.Linear(self.hid_dim, self.output_dim)
        self.dropout = nn.Dropout(self.dropout)

        self.pad_idx = config.pad_idx


    def energy_init(self):
      for n_layer, layer in enumerate(self.decoder.layers):
        layer.self_attention.energy_init()


    def forward(self, tokens):
      st_time = time.time()

      src_len = tokens.shape[1]
      batch_size = tokens.shape[0]

      logits = torch.zeros(batch_size, src_len, self.output_dim).to(self.device)
      hidden_states = torch.zeros(batch_size, src_len, self.n_layer, self.hid_dim).to(self.device)
      for cur_idx in range(src_len):
        tok = tokens[:, cur_idx]
        hidden_state = self.embeddings(tok).view(-1, self.hid_dim)

        hidden_state, out_hidden_states = self.decoder(hidden_state)

        logits[:, cur_idx] = self.logits(hidden_state)
        hidden_states[:, cur_idx] = out_hidden_states

      outputs = (logits, hidden_states)
      if logits.isnan().any() or logits.isinf().any():
        print('OUTPUT NAN INF = True')

      print("--- %s ITERATION TIME ---" % (time.time() - st_time))
      return outputs

In [ ]:
class DecoderMM(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.device = config.device

        #self.reinforce = nn.Embedding(2, hid_dim) # create embeddings for intent
        self.hid_dim = config.hid_dim
        self.cortex_dim = config.cortex_dim
        self.n_layer = config.n_layer
        self.n_head = config.n_head
        self.hid_dim = config.hid_dim

        self.layers = nn.ModuleList([InterpreterLayer(config)
                                     for _ in range(self.n_layer)])

        #self.init_intent = nn.Linear(1, self.hid_dim)
        self.output_dim = config.output_dim

        self.dropout = config.dropout
        #self.intent = None
        #self.pos_mask = nn.Parameter(torch.ones(hid_dim, requires_grad=True))
        self.head_dim = int(self.cortex_dim / self.n_head)


        self.ln_f = nn.LayerNorm(self.hid_dim)
        self.dropout = nn.Dropout(self.dropout)


    def forward(self, emb):
      batch_size = emb.shape[0]
      #att_output = torch.zeros(batch_size, self.n_layer, config.hid_dim).to(self.device)
      #if self.intent is None:
      # self.intent = self.init_intent(torch.ones(1).to(self.device)).repeat(batch_size, 1)
      hidden_states = torch.zeros(batch_size, self.n_layer, self.hid_dim).to(self.device)

      intent = self.dropout(emb)
      for n_layer, layer in enumerate(self.layers):
          intent, attention = layer(intent, train=True)
          hidden_states[:, n_layer] = intent

      intent = self.ln_f(intent)
      return intent, hidden_states

In [ ]:
class SynapticPlasticityModel(nn.Module):
    def __init__(self, config):
        super().__init__()

        self.device = config.device
        self.is_bias = config.is_bias
        self.dropout = config.dropout
        #self.n_head = config.n_head
        #self.hid_dim = config.hid_dim
        self.hid_dim = config.hid_dim
        self.synapse_static_dim = config.synapse_static_dim
        self.synapse_dynamic_dim = config.synapse_dynamic_dim

        self.neuron_static_dim = config.neuron_static_dim
        self.neuron_dynamic_dim = config.neuron_dynamic_dim
        #self.synapse_dim_2 = config.get('synapse_dim_2', None)
        #assert self.hid_dim % self.n_head == 0
        #self.head_dim = int(self.hid_dim / self.n_head)

        self.dynamic_layers = nn.ModuleList([DynamicLayer(config) for _ in range(self.n_layer)])
        self.static_layer = StaticLayer(config)

        #self.to_global_query = nn.Linear(1, self.synapse_dim, bias = self.is_bias)
        #self.to_local_query = nn.Linear(1, self.synapse_dim, bias = self.is_bias)

        self.to_fAP = nn.Linear(self.neuron_static_dim, self.synapse_static_dim, bias = self.is_bias)
        self.to_bAP = nn.Linear(self.neuron_static_dim, self.synapse_static_dim, bias = self.is_bias)

        self.dropout = nn.Dropout(self.dropout)
        self.activation = nn.Sigmoid()
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)


    def forward(self, hidden_state, mask = None):
        batch_size = query.shape[0]
        hidden_states = torch.zeros(batch_size, self.n_layer, self.hid_dim).to(self.device)

        for n_layer, dynamic_layer in enumerate(self.dynamic_layers):
          hidden_state = hidden_state.view(-1, self.hid_dim, 1)

          static_layer.to_neuron_state(hidden_state)
          dynamic_layer.to_neuron_state(hidden_state)
          #static_layer.neuron_state = static_layer.neuron_state + static_layer.to_neuron_state(hidden_state)
          #dynamic_layer.neuron_state = dynamic_layer.neuron_state + dynamic_layer.to_neuron_state(hidden_state)

          act_neuron_static_state = self.activation(static_layer.neuron_state)
          act_neuron_dynamic_state = self.activation(dynamic_layer.neuron_state)

          act_synapse_static_state = self.activation(static_layer.synapse_state)
          act_synapse_dynamic_state = self.activation(dynamic_layer.synapse_state)

          query = static_layer.to_query(act_neuron_static_state) + dynamic_layer.to_query(act_neuron_dynamic_state)
          hidden_state = static_layer.to_out(act_neuron_static_state) + dynamic_layer.to_out(act_neuron_dynamic_state)
          hidden_state = self.mlp(hidden_state.view(-1, self.hid_dim))
          #local_query = self.to_local_query(act_neuron_dynamic_state).view(-1, self.hid_dim, self.synapse_dim)

          fAP = self.to_fAP(act_neuron_static_state).view(-1, self.hid_dim, 1, self.synapse_dim)
          bAP = self.to_bAP(act_neuron_static_state).view(-1, self.hid_dim, 1, self.synapse_dim).permute(0, 2, 1, 3)

          AP = fAP + bAP

          dynamic_layer.synapse_state = static_layer.to_synapse_dynamic_state(act_synapse_static_state) + dynamic_layer.to_synapse_dynamic_state(act_synapse_dynamic_state) + dynamic_layer.synapse_state
          static_layer.synapse_state = dynamic_layer.to_synapse_static_state(act_synapse_dynamic_state) + static_layer.to_synapse_static_state(act_synapse_static_state) + AP + static_layer.synapse_state

          static_value = torch.einsum( 'ijl, ijkl->ikl', static_query, static_layer.synapse_state)
          #local_value = torch.einsum( 'ijl, ijkl->ikl', local_query, self.synapse_dynamic_state)

          dynamic_layer.neuron_state = static_layer.to_neuron_dynamic_state(act_neuron_static_state) + dynamic_layer.to_neuron_dynamic_state(act_neuron_dynamic_state) + dynamic_layer.neuron_state
          static_layer.neuron_state = dynamic_layer.to_neuron_static_state(act_neuron_dynamic_state) + static_layer.to_neuron_static_state(act_neuron_static_state) + static_value + static_layer.neuron_state

          hidden_states[:, n_layer] = intent

        return out

In [ ]:
class DynamicLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.device = config.device
        self.is_bias = config.is_bias
        self.hid_dim = config.hid_dim
        self.synapse_static_dim = config.synapse_static_dim
        self.synapse_dynamic_dim = config.synapse_dynamic_dim
        self.neuron_static_dim = config.neuron_static_dim
        self.neuron_dynamic_dim = config.neuron_dynamic_dim
        self.query_dim = config.query_dim
        self.dropout = config.dropout

        self.synapse_state = None
        self.neuron_state = None

        self.synapse_state_init = nn.Parameter(torch.randn(1, self.hid_dim, self.hid_dim, self.synapse_static_dim))
        self.neuron_state_init = nn.Parameter(torch.randn(1, self.hid_dim, self.hid_dim, self.neuron_static_dim))

        self.to_synapse_dynamic_state = nn.Linear(self.neuron_static_dim, self.neuron_dynamic_dim, bias = self.is_bias)
        self.to_synapse_static_state = nn.Linear(self.neuron_static_dim, self.neuron_static_dim, bias = self.is_bias)

        self.to_neuron_state = nn.Linear(1, self.synapse_static_dim, bias = self.is_bias)
        self.to_hidden_state = nn.Parameter(torch.randn(self.synapse_static_dim, 1))

        self.to_neuron_static_state = nn.Linear(self.synapse_static_dim, self.synapse_static_dim, bias = self.is_bias)
        self.to_neuron_dynamic_state = nn.Linear(self.synapse_static_dim, self.synapse_dynamic_dim, bias = self.is_bias)
        self.to_query = nn.Linear(self.neuron_static_dim, self.query_dim, bias = self.is_bias)
        self.to_out = nn.Linear(self.neuron_static_dim, self.query_dim, bias = self.is_bias)

        self.dropout = nn.Dropout(self.dropout)


    def init_params(self):
        self.synapse_state = self.synapse_state_init + 0
        self.neuron_state = self.neuron_state_init + 0



In [ ]:
class StaticLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.device = config.device
        self.is_bias = config.is_bias
        self.hid_dim = config.hid_dim
        self.synapse_static_dim = config.synapse_static_dim
        self.synapse_dynamic_dim = config.synapse_dynamic_dim
        self.neuron_static_dim = config.neuron_static_dim
        self.neuron_dynamic_dim = config.neuron_dynamic_dim
        self.query_dim = config.query_dim
        self.dropout = config.dropout

        self.synapse_state = None
        self.neuron_state = None

        self.synapse_state_init = nn.Parameter(torch.randn(1, self.hid_dim, self.hid_dim, self.synapse_static_dim))
        self.neuron_state_init = nn.Parameter(torch.randn(1, self.hid_dim, self.hid_dim, self.neuron_static_dim))

        self.to_synapse_dynamic_state = nn.Linear(self.neuron_static_dim, self.neuron_dynamic_dim, bias = self.is_bias)
        self.to_synapse_static_state = nn.Linear(self.neuron_static_dim, self.neuron_static_dim, bias = self.is_bias)

        self.to_neuron_static_state = nn.Linear(self.synapse_static_dim, self.synapse_static_dim, bias = self.is_bias)
        self.to_neuron_dynamic_state = nn.Linear(self.synapse_static_dim, self.synapse_dynamic_dim, bias = self.is_bias)
        self.to_neuron_state = nn.Linear(1, self.synapse_static_dim, bias = self.is_bias)

        self.to_query = nn.Linear(self.neuron_static_dim, self.query_dim, bias = self.is_bias)
        self.to_out = nn.Linear(self.neuron_static_dim, self.query_dim, bias = self.is_bias)

        self.dropout = nn.Dropout(self.dropout)


    def init_params(self):
        self.synapse_state = self.synapse_state_init + 0
        self.neuron_state = self.neuron_state_init + 0


In [ ]:
class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dropout = config.dropout
        self.hid_dim = config.hid_dim
        self.pf_dim = config.pf_dim
        self.fc_1 = nn.Linear(self.hid_dim, self.pf_dim)
        self.fc_2 = nn.Linear(self.pf_dim, self.hid_dim)
        self.activation = nn.GELU()
        self.dropout = nn.Dropout(self.dropout)

    def forward(self, x):
        x = self.dropout(self.activation(self.fc_1(x)))
        x = self.fc_2(x)
        return x


In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()

        assert hid_dim % n_heads == 0

        self.hid_dim = hid_dim                 # total hidden dimention of all attention heads together
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        self.dropout = nn.Dropout(dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]

        # calculate query, key, value matrices
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]

        # split into multiple heads of self attention and permute
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)

        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]

        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale

        #energy = [batch size, n heads, query len, key len]

        # ignore pad-tokens or future predictions if we have a mask to do so
        #   initially we ignore them in energy = (Q x K^T) / sqrt(dim)
        #   but this also ignores them when multiplying energy by V while geting new x
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)

        # [batch size, 1, trg len, trg len]

        attention = torch.softmax(energy, dim = -1)

        #attention = [batch size, n heads, query len, key len]

        x = torch.matmul(self.dropout(attention), V)

        #x = [batch size, n heads, query len, head dim]

        # permute not just the references to memory, but also permute in memory
        x = x.permute(0, 2, 1, 3).contiguous()

        #x = [batch size, query len, n heads, head dim]

        x = x.view(batch_size, -1, self.hid_dim)

        #x = [batch size, query len, hid dim]

        # calculate the weighted sum of what we got from the attention heads
        x = self.fc_o(x)

        #x = [batch size, query len, hid dim]

        return x, attention

## GLOBAL TRAINING PARAMETERS

In [ ]:
def count_parameters(model):
    count = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(count)
    return count


def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)


def freeze_embeddings(model, req_grad=False):
  embeddings = model.embeddings
  for c_p in embeddings.parameters():
      c_p.requires_grad = req_grad


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs



## MY_MODEL LEARNING

### LEARNING FUNC

In [ ]:
def save_state(epoch, n_batch, model, optimizer, PATH):
  torch.save({
              'epoch': epoch,
              'n_batch': n_batch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              }, PATH)

In [ ]:
def train(epoch, optimizer, model, dataloader, clip, last_batch=0, train_history=None, valid_history=None, is_ltm_update=True, training = False, max_len=200):


    def print_ram_state():
      split_bar = '='*20
      memory_info = psutil.virtual_memory()._asdict()
      print(f"{split_bar} Memory Usage {split_bar}")
      for k,v in memory_info.items():
        print(k, v)


    def split_batch(batch, path_len):
      chunks = math.ceil(batch.shape[1]/path_len)
      list_data = list(torch.chunk(batch, chunks, dim=1))
      print(len(list_data))

      for i, item in enumerate(list_data):
        if i < len(list_data)-1:
          item = torch.cat((item, list_data[i+1][:, 0].view(-1, 1)), dim=1)
          list_data[i] = item

      return list_data

    model.train()
    epoch_loss = 0
    history = []
    time_it = []
    start_time = time.time()
    LEARNING_RATE = 5e-4 # twice lower than the Adam's default 1e-3

    is_lm_learn = False
    PATH_LEN = max_len

    print(f'LAST BATCH - {last_batch}')
    for i, batch in enumerate(dataloader):
      isNaN = 0
      if i < last_batch:
        continue
      model.clear()
      batch = batch.to(device)
      print(f'BATCH - {i}')
      print(f'SRC LEN - {batch.shape[1]}')
      mask = get_mask(batch)
      list_src = split_batch(batch, PATH_LEN)
      list_mask = split_batch(mask, PATH_LEN)

      for id, src in enumerate(list_src):
        if isNaN:
          continue
        gc.collect()
        torch.cuda.empty_cache()

        src = src.to(device)
        model = model.to(device)


        optimizer.zero_grad()
        st_time = time.time()
        outputs = model(src, inf=False, training=training, is_ltm_update=is_ltm_update)
        output_dim = outputs.shape[-1]
        outputs = outputs[:,:-1].contiguous().view(-1, output_dim)
        trg = src#list_mask[id]
        trg = trg[:,1:].contiguous().view(-1) # ignore <sos>
        loss_n = criterion(outputs, trg)
        if loss_n.isnan().any():
          isNaN = True
          model.detach_kernel()
          continue
        model.detach_kernel()
        loss_n.backward(retain_graph=False)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        print(f'sub-loss = {loss_n.item()}')
        loss = float(loss_n) #loss_n.detach()

        epoch_loss += loss
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        history.append(loss)

      if (i+1)%10==0:
          save_state(epoch, i, model, optimizer, PATH)
          fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 8))

          clear_output(True)
          ax[0].plot(history, label='train loss')
          ax[0].set_xlabel('Batch')
          ax[0].set_title('Train loss')
          if train_history is not None:
              ax[1].plot(train_history, label='general train history')
              ax[1].set_xlabel('Epoch')
          if valid_history is not None:
              ax[1].plot(valid_history, label='general valid history')
          plt.legend()

          plt.show()
    return epoch_loss / len(src)

### dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset('wikitext', 'wikitext-103-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from torch.nn.utils.rnn import pad_sequence
from tokenizers import Tokenizer

In [ ]:
PAD_ID = tokenizer.unk_token_id

In [ ]:
def tokenization(example):
    return tokenizer(example["text"])

dataset = dataset['train'].map(tokenization, batched=True)

Map:   0%|          | 0/1801350 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1059 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

In [ ]:
def collate_tokenize(data):
  text_batch = [element["input_ids"] for element in data]
  batch = pad_sequence(text_batch, padding_value=PAD_ID, batch_first=True)
  return batch

In [ ]:
from torch.utils.data import DataLoader
dataloader = DataLoader(dataset, batch_size=3, collate_fn=collate_tokenize)
  #print(batch)

### localh

In [ ]:
# for mm
INPUT_DIM = tokenizer.vocab_size
HID_DIM = trg_model.config.n_embd
hid_dim = 100
DROPOUT = 0.1
N_HEAD = trg_model.config.n_head
N_LAYER = trg_model.config.n_layer
synapse_dim = 4
synapse_dim_2 = 4

max_grad_norm = 1.0
LAYER_DEPTH = 4
HID_INT_DIM = trg_model.config.n_ctx
HID_GEN_DIM = trg_model.config.n_ctx
is_bias = False
hid_s_layer = 5
n_depth = 2
k_size = 40

config = {
    'hid_dim': HID_DIM,
    'hid_dim': hid_dim,
    'cortex_dim': HID_INT_DIM,
    'output_dim': INPUT_DIM,
    'dropout': DROPOUT,
    'n_head': N_HEAD,
    'n_layer': N_LAYER,
    'is_bias': is_bias,
    'layer_depth': LAYER_DEPTH,
    'synapse_dim': synapse_dim,
    'synapse_dim_2': synapse_dim_2,
    'hid_s_layer': hid_s_layer,
    'n_depth': n_depth,
    'k_size': k_size,
    'device': device,
    'pad_idx': PAD_ID
}

### Init model

In [ ]:
decoderMM = DecoderMM(config)

#ML = MemoryLoss()
MSE = nn.MSELoss()
src_model = ModelMM(decoderMM, config)

In [ ]:
src_model.apply(initialize_weights);
print(f'The modelMM has {count_parameters(src_model):,} trainable parameters')

109144081
The modelMM has 109,144,081 trainable parameters


In [ ]:
LEARNING_RATE = 5e-4 # twice lower than the Adam's default 1e-3
optimizer = torch.optim.Adam(src_model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_ID)